##### Copyright 2019 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Сохранение и восстановление модели с использованием распеделенной стратегии

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/distribute/save_and_load"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/distribute/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/distribute/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Изучайте код на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/distribute/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачайте ноутбук</a>
  </td>

</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

## Введение

Часто во время обучения модель сохраняют и восстанавливают. Существует два набора API-интерфейсов для сохранения и загрузки моделей keras: API высокого уровня и API низкого уровня. В этом руководстве показано, как можно использовать `SavedModel` в контексте `tf.distribute.Strategy`. Чтобы узнать о `SavedModel` и сериализации в целом, прочтите [руководство по сохраненению моделей](../../guide/saved_model.ipynb) и [руководство по сериализации модели Keras](../../guide/keras/save_and_serialize.ipynb). Начнем с простого примера:

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

Подготовка данных и модели для использования с `tf.distribute.Strategy`:

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()

def get_data():
  datasets, ds_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
  mnist_train, mnist_test = datasets['train'], datasets['test']

  BUFFER_SIZE = 10000

  BATCH_SIZE_PER_REPLICA = 64
  BATCH_SIZE = BATCH_SIZE_PER_REPLICA * mirrored_strategy.num_replicas_in_sync

  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255

    return image, label

  train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
  eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

  return train_dataset, eval_dataset

def get_model():
  with mirrored_strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10)
    ])

    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=[tf.metrics.SparseCategoricalAccuracy()])
    return model

Обучение модели: 

In [ ]:
model = get_model()
train_dataset, eval_dataset = get_data()
model.fit(train_dataset, epochs=2)

## Сохранение и загрузка модели

Сейчас у вас есть простая модель. Давайте посмотрим на API сохранения/загрузки. 
Вам доступно два вида API:

*   Высокоуровневое API keras `tf.keras.model.save` и `tf.keras.models.load_model`
*   Низкоуровневое `tf.saved_model.save` и `tf.saved_model.load`


### Keras API

Вот пример сохранения и загрузки модели с помощью API Keras:

In [ ]:
keras_model_path = "/tmp/keras_save"
model.save(keras_model_path)

Восстановление модели без использования `tf.distribute.Strategy`:

In [ ]:
restored_keras_model = tf.keras.models.load_model(keras_model_path)
restored_keras_model.fit(train_dataset, epochs=2)

После восстановления модели вы можете продолжить ее обучение, даже без повторного вызова `compile()`, поскольку она уже скомпилирована перед сохранением. Модель сохраняется в стандартном формате TensorFlow `SavedModel`. Для получения дополнительной информации обратитесь к [руководству по формату `saved_model`](../../guide/saved_model.ipynb).

Теперь загрузим модель и обучим ее с помощью `tf.distribute.Strategy`:

In [ ]:
another_strategy = tf.distribute.OneDeviceStrategy("/cpu:0")
with another_strategy.scope():
  restored_keras_model_ds = tf.keras.models.load_model(keras_model_path)
  restored_keras_model_ds.fit(train_dataset, epochs=2)

Как вы можете видеть, загрузка работает точно также и с `tf.distribute.Strategy`. Стратегия используемая для восстановления не обязательно должна совпадать со стратегией, использованной до сохранения.

### API `tf.saved_model` 

Теперь давайте посмотрим на API нижнего уровня. Сохранение модели аналогично keras API:

In [ ]:
model = get_model()  # получаем новую модель
saved_model_path = "/tmp/tf_save"
tf.saved_model.save(model, saved_model_path)

Загрузка может быть произведена с помощью `tf.saved_model.load()`. Однако, поскольку это API нижнего уровня (и, следовательно, имеет более широкий диапазон вариантов использования), он не возвращает модель Keras. Вместо этого он возвращает объект, содержащий функции, которые можно использовать для вывода. Например:

In [ ]:
DEFAULT_FUNCTION_KEY = "serving_default"
loaded = tf.saved_model.load(saved_model_path)
inference_func = loaded.signatures[DEFAULT_FUNCTION_KEY]

Загруженный объект может содержать несколько функций, каждая из которых связана с ключом. `"serving_default"` - это ключ по умолчанию для функции вывода с сохраненной моделью Keras. Чтобы сделать вывод с помощью этой функции:

In [ ]:
predict_dataset = eval_dataset.map(lambda image, label: image)
for batch in predict_dataset.take(1):
  print(inference_func(batch))

Вы также можете загружать и выполнять вывод распределенным способом:

In [ ]:
another_strategy = tf.distribute.MirroredStrategy()
with another_strategy.scope():
  loaded = tf.saved_model.load(saved_model_path)
  inference_func = loaded.signatures[DEFAULT_FUNCTION_KEY]

  dist_predict_dataset = another_strategy.experimental_distribute_dataset(
      predict_dataset)

  # вызов функции распределенным способом
  for batch in dist_predict_dataset:
    another_strategy.run(inference_func,args=(batch,))

Вызов восстановленной функции - это просто прямой переход к сохраненной модели(прогнозу). Что, если вы не хотите продолжать обучение загруженной функции? Или встроить загруженную функцию в более крупную модель? Обычной практикой является перенос этого загруженного объекта в слой Keras для достижения этой цели. К счастью, [TF Hub](https://www.tensorflow.org/hub) имеет для этой цели [hub.KerasLayer](https://github.com/tensorflow/hub/blob/master/tensorflow_hub/keras_layer.py)

In [ ]:
import tensorflow_hub as hub

def build_model(loaded):
  x = tf.keras.layers.Input(shape=(28, 28, 1), name='input_x')
  # Wrap what's loaded to a KerasLayer
  keras_layer = hub.KerasLayer(loaded, trainable=True)(x)
  model = tf.keras.Model(x, keras_layer)
  return model

another_strategy = tf.distribute.MirroredStrategy()
with another_strategy.scope():
  loaded = tf.saved_model.load(saved_model_path)
  model = build_model(loaded)

  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])
  model.fit(train_dataset, epochs=2)

Как видите, `hub.KerasLayer` переносит результат, загруженный из `tf.saved_model.load()`, в слой Keras, который можно использовать для построения другой модели. Это очень полезно для трансферного обучения.

### Какой API мне следует использовать?

Если вы работаете с моделью keras, для сохранения, почти всегда рекомендуется использовать API Keras `model.save()`. Если же то, что вы сохраняете, не является моделью Keras, тогда ваш единственный выбор - API низкого уровня.

Какой API вы используете для загрузки, зависит от того, что вы хотите получить от API загрузки. Если вы не можете (или не хотите) получить модель Keras, используйте `tf.saved_model.load()`. В противном случае используйте `tf.keras.models.load_model()`. Обратите внимание, что вы можете восстановить модель Keras, только если вы сохранили модель Keras.

Можно смешивать API. Вы можете сохранить модель Keras с помощью `model.save()` и загрузить модель, отличную от Keras, с помощью низкоуровневого API `tf.saved_model.load()`.

In [ ]:
model = get_model()

# Сохраняем модель, исользуя API Keras save() 
model.save(keras_model_path) 

another_strategy = tf.distribute.MirroredStrategy()
# Загружаем модель, используя низкоуровневое API
with another_strategy.scope():
  loaded = tf.saved_model.load(keras_model_path)

### Сохранение/загрузка с локального устройства

При сохранении и загрузке с локального устройства при удаленной работе, например, с использованием облачного TPU, необходимо использовать опцию `experimental_io_device`, чтобы установить устройство ввода-вывода на localhost-е.

In [ ]:
model = get_model()

# Сохраняем модель на localhost.
saved_model_path = "/tmp/tf_save"
save_options = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
model.save(saved_model_path, options=save_options)

# Загружаем модель с localhost-а.
another_strategy = tf.distribute.MirroredStrategy()
with another_strategy.scope():
  load_options = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
  loaded = tf.keras.models.load_model(saved_model_path, options=load_options)

### Предостережения

Особый случай - это когда у вас есть модель Keras, которая не имеет четко определенных входных данных. Например, последовательная модель может быть создана без какой-либо размерности входных данных (`Sequential ([Dense (3), ...]`). Модели наследования также не имеют четко определенных входных данных после инициализации. В этом случае вам следует использовать низкоуровневое API, как при сохранении, так и при загрузке, иначе вы получите ошибку.

Чтобы проверить, имеет ли ваша модель четко определенные входные данные, просто проверьте, имеет ли `model.inputs` значение `None`. Если это не `None`, то все в порядке. Размерность входных данных определяется автоматически, когда модель использует `.fit`, `.evaluate`, `.predict` или при вызове модели с передачей входа `model(inputs)`.

Вот пример:

In [ ]:
class SubclassedModel(tf.keras.Model):

  output_name = 'output_layer'

  def __init__(self):
    super(SubclassedModel, self).__init__()
    self._dense_layer = tf.keras.layers.Dense(
        5, dtype=tf.dtypes.float32, name=self.output_name)

  def call(self, inputs):
    return self._dense_layer(inputs)

my_model = SubclassedModel()
# my_model.save(keras_model_path)  # ERROR! 
tf.saved_model.save(my_model, saved_model_path)